In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-2a', mod, True, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
0,ABNB,RROnly,945.29,906.57,1851.86,85.997514,0.952381,147.60,124.019997,-0.16,9.226190
71,U,RROnly,622.89,1129.60,1752.49,24.007653,0.943038,33.82,25.959999,-0.23,9.182719
69,TTD,RROnly,862.85,646.15,1509.00,16.647744,0.941489,71.31,56.779999,-0.20,9.122340
27,GFS,RROnly,241.67,284.48,526.15,40.062603,0.950980,55.94,49.660000,-0.11,9.064171
67,TOST,RROnly,98.17,213.09,311.26,13.382885,0.935185,19.28,15.800000,-0.18,8.991770
33,INTA,RROnly,188.46,177.16,365.61,14.489806,0.941176,41.50,35.500000,-0.14,8.991597
39,MDB,RROnly,4242.31,3926.65,8168.96,109.363651,0.920213,500.90,369.070007,-0.26,8.895254
11,CRWD,RROnly,2022.68,1410.69,3433.37,38.146395,0.925532,329.34,262.380005,-0.20,8.882979
63,SQ,RROnly,1578.32,1059.51,2637.83,42.139421,0.920213,66.88,81.959999,0.23,8.855227
36,JKS,RROnly,503.73,518.43,1022.16,13.703220,0.920213,26.19,20.410000,-0.22,8.839458


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-1a', mod, False)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [11]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
33,XLB,RROnly,315.91,234.28,550.19,36.236671,0.962766,85.17,75.720001,-0.11,9.289168
9,IJR,RROnly,448.95,337.19,786.14,47.200766,0.957447,107.38,98.040001,-0.09,9.101655
24,SMOG,RROnly,618.95,712.17,1331.12,50.917566,0.930851,96.98,78.379997,-0.19,8.944569
3,DAPP,RROnly,81.84,135.29,217.12,2.945213,0.923664,10.30,7.760000,-0.25,8.931298
7,FPX,RROnly,451.25,501.32,952.57,55.472600,0.952128,100.44,91.910004,-0.08,8.922872
32,XITK,RROnly,1054.35,953.99,2008.33,86.318980,0.930851,151.57,130.419998,-0.14,8.814590
44,XME,RROnly,351.10,196.77,547.88,14.344376,0.936170,56.66,50.520000,-0.11,8.781431
11,IXC,RROnly,175.74,85.49,261.23,11.198714,0.941489,39.30,35.779999,-0.09,8.764775
18,MGK,RROnly,994.43,736.36,1730.79,112.711793,0.941489,279.15,253.199997,-0.09,8.764775
8,ICLN,RROnly,130.68,105.28,235.96,8.294507,0.909574,14.51,10.940000,-0.25,8.734043


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'AMSC', 'ACHR',  'APPS', 'AUGX', 'AUR',   'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FSR',
            'GOVX', 'GRAB', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'NNOX', 'MAX', 'MTTR',
            'NEPH',  'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SOUN', 'SST', 'STEM', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-2b', mod, True)[0]

In [26]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0) & (df3.Last > 0.1)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
57,TRUP,RROnly,876.42,726.03,1602.44,22.229006,0.920213,22.88,32.110001,0.40,5.213548
41,OPEN,RROnly,150.38,169.44,319.82,1.475140,0.918605,3.00,3.770000,0.26,5.090498
22,FSR,RROnly,157.00,155.75,312.75,1.273509,0.872340,0.73,1.080000,0.48,4.865457
3,APPS,RROnly,455.22,456.60,911.82,5.062754,0.888298,3.48,4.500000,0.29,4.840102
51,SABR,RROnly,62.57,85.88,148.45,3.217803,0.872340,2.96,4.080000,0.38,4.795468
10,BSGM,RROnly,215.17,518.21,733.38,2.286100,0.819149,0.62,1.760000,1.83,4.719697
23,GOVX,RROnly,327.11,675.87,1002.98,4.404379,0.816456,2.35,5.690000,1.42,4.673424
45,PTON,RROnly,630.51,717.10,1347.61,6.601657,0.840426,4.54,6.900000,0.52,4.636806
18,DPSI,RROnly,38.31,28.76,67.07,4.053021,0.948052,7.55,7.910000,0.05,4.537815
20,EXPR,RROnly,742.58,831.68,1574.26,5.538347,0.797872,2.83,5.530000,0.96,4.482817


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-1b', mod4, True)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
34,TSLA,RROnly,2950.31,2356.97,5307.28,30.741448,0.936170,212.19,258.170013,0.22,6.396795
37,UPS,RROnly,835.44,807.65,1643.09,78.830785,0.952128,156.89,169.690002,0.08,6.202508
26,NKE,RROnly,670.49,607.76,1278.25,62.279772,0.936170,101.78,114.570000,0.13,6.195932
27,NVDA,RROnly,2966.18,1566.79,4532.98,55.303918,0.925532,594.91,503.529999,-0.15,6.114488
6,BA,RROnly,1699.80,1364.70,3064.50,99.376621,0.920213,215.02,245.720001,0.14,6.003040
10,CDNS,RROnly,1196.14,598.53,1794.67,57.219708,0.925532,292.14,261.739990,-0.10,5.866261
3,AMD,RROnly,865.49,752.23,1617.72,41.034336,0.904255,174.23,148.500000,-0.15,5.820669
7,BKNG,RROnly,15419.41,10277.24,25696.65,1190.424888,0.930851,3616.51,3326.540039,-0.08,5.784574
1,ACN,RROnly,1486.79,1042.57,2529.36,139.050120,0.925532,363.90,332.000000,-0.09,5.783519
0,AAPL,RROnly,1031.61,483.00,1514.62,57.047958,0.920213,191.56,174.929993,-0.09,5.686424


In [ ]:
findata.EPOCHS=200
tickers5 = [ 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CDUAF'. 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EDNMF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-12b', mod5, False)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
58,SE,RROnly,1672.30,1205.63,2877.93,40.284948,0.898936,39.57,64.120003,0.62,6.094960
51,PROSF,RROnly,532.56,512.86,1045.41,28.799596,0.877660,30.51,43.520000,0.43,5.699972
1,ALSMY,RROnly,23.93,25.66,49.59,1.209417,0.898936,1.23,1.490000,0.21,5.458460
40,LVMUY,RROnly,704.40,713.75,1418.15,60.728020,0.904255,167.75,137.220001,-0.18,5.395137
47,ONON,RROnly,120.75,177.05,297.80,16.685815,0.906542,26.25,30.490000,0.16,5.307076
41,MYTE,RROnly,39.30,133.05,172.35,2.521267,0.870504,2.99,3.830000,0.28,5.292909
39,LYB,RROnly,440.85,429.64,870.50,28.969300,0.946809,95.43,88.779999,-0.07,5.243161
55,SFTBY,RROnly,156.49,187.10,343.59,13.380239,0.909574,22.09,25.230000,0.14,5.205167
68,VALE,RROnly,98.80,75.75,174.55,4.933374,0.898936,14.17,16.370001,0.16,5.157675
15,CX,RROnly,42.77,31.04,73.81,1.761887,0.893617,8.24,6.810000,-0.17,5.131414


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-1b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
0,ARGT,RROnly,227.29,111.80,339.09,14.221951,0.877660,53.66,44.959999,-0.16,8.011968
12,EWZ,RROnly,183.33,103.24,286.57,16.100103,0.930851,33.37,32.169998,-0.04,7.579787
10,EWW,RROnly,241.33,136.18,377.51,23.554016,0.867021,66.39,58.869999,-0.11,7.461315
17,TUR,RROnly,169.07,127.00,296.07,16.369017,0.904255,35.29,37.310001,0.06,7.446809
13,IDX,RROnly,67.49,41.37,108.87,9.760697,0.882979,15.91,17.230000,0.08,7.367021
5,EWA,RROnly,80.26,55.30,135.56,11.159421,0.920213,23.69,22.650000,-0.04,7.207447
16,THD,RROnly,272.41,221.84,494.25,44.778693,0.952128,59.69,60.669998,0.02,7.127660
15,KEMQ,RROnly,72.20,137.42,209.63,11.908489,0.835106,13.66,14.910000,0.09,6.518913
8,EWM,RROnly,54.54,63.49,118.03,16.692549,0.914894,21.12,20.410000,-0.03,6.312057
18,XCEM,RROnly,92.70,54.35,147.05,16.022655,0.867021,29.52,27.959999,-0.05,6.010638


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BLD', 'BLDR', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SCCO', 'SCL', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-12b', mod7, False)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
89,TGT,RROnly,1150.53,1052.60,2203.14,86.665261,0.962766,145.49,125.839996,-0.14,9.361702
88,TEX,RROnly,391.46,281.72,673.18,13.013144,0.952128,63.36,53.400002,-0.16,9.222074
15,BXC,RROnly,757.59,576.39,1333.98,5.717371,0.941489,117.35,92.889999,-0.21,9.136272
1,ADI,RROnly,943.68,502.35,1446.03,78.858272,0.946809,192.10,170.679993,-0.11,8.984526
22,CI,RROnly,1152.33,838.87,1991.20,126.295136,0.936170,323.84,278.859985,-0.14,8.905775
73,PXD,RROnly,1236.56,763.00,1999.56,48.199477,0.936170,227.47,200.330002,-0.12,8.829787
68,PETS,RROnly,139.37,173.15,312.51,6.651400,0.909574,6.10,7.620000,0.25,8.734043
77,SAFE,RROnly,486.50,543.16,1029.65,16.369689,0.920213,20.68,23.690001,0.15,8.670213
65,PARA,RROnly,249.68,426.18,675.86,11.098642,0.888298,14.43,18.150000,0.26,8.453355
99,WSM,RROnly,1134.96,803.81,1938.77,31.380363,0.930851,200.34,184.149994,-0.08,8.444149


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-2a', mod8, True)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
3,SLV,RROnly,97.61,88.67,186.28,11.663038,0.861702,20.75,24.129999,0.16,7.752660
9,WEAT,RROnly,23.67,34.35,58.02,5.061192,0.744681,5.72,5.260000,-0.08,4.255319
4,SOYB,RROnly,74.40,54.80,129.20,13.745025,0.797872,25.06,24.379999,-0.03,1.241135
0,GLD,RROnly,400.04,353.28,753.32,139.719432,0.888298,188.61,190.899994,0.01,0.000000
1,ISHG,RROnly,72.51,95.90,168.41,63.643567,0.819149,70.07,69.339996,-0.01,0.000000
2,PDBC,RROnly,45.83,26.88,72.71,6.410962,0.845745,13.18,13.130000,-0.00,0.000000
5,TIP,RROnly,115.65,94.59,210.25,94.680229,0.835106,107.00,105.449997,-0.01,0.000000
6,TLT,RROnly,208.71,429.77,638.48,83.598186,0.840426,96.07,95.010002,-0.01,0.000000
7,UUP,RROnly,33.69,23.46,57.15,22.591361,0.813830,27.94,27.379999,-0.02,0.000000
8,VTIP,RROnly,29.14,9.10,38.25,40.277118,0.856383,47.47,47.310001,-0.00,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TRX-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-1a', mod9, False)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
1,AVAX-USD,RROnly,823.72,490.93,1314.65,12.387587,0.787234,38.16,28.860001,-0.24,4.736575
8,MATIC-USD,RROnly,10.97,13.08,24.05,0.439813,0.739362,0.85,0.690000,-0.19,3.909375
0,ADA-USD,RROnly,13.16,9.34,22.50,0.299319,0.781915,0.54,0.480000,-0.13,3.907529
2,BTC-USD,RROnly,222102.79,181862.48,403965.27,17261.937856,0.813830,47270.67,44088.769531,-0.07,3.153495
7,LTC-USD,RROnly,798.59,1064.93,1863.52,53.016735,0.781915,70.56,66.419998,-0.06,1.950355
10,TRX-USD,RROnly,0.32,-0.01,0.31,0.057620,0.553191,0.12,0.100000,-0.21,1.823708
5,ETH-USD,RROnly,21322.88,23017.06,44339.94,1073.911781,0.882979,2493.29,2435.870117,-0.02,0.455927
3,DOGE-USD,RROnly,1.29,1.05,2.34,0.066539,0.675532,0.08,0.080000,-0.03,0.000000
4,DOT-USD,RROnly,162.96,176.39,339.36,4.607876,0.728723,7.20,6.970000,-0.03,0.000000
6,LINK-USD,RROnly,99.72,127.59,227.31,6.926631,0.670213,18.41,19.570000,0.06,0.000000


In [27]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
etf-11a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-11a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-11a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
etf-11a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
mcap-11b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
mcap-11b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
mcap-11b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
mcap-11b-test-low-sh-1-s